---
title: Core
subtitle: Core data structures and algorithms
---


In [ ]:
# | default_exp core
# | export
from pydantic import BaseModel, ConfigDict, Field
from datetime import datetime, timedelta

In [ ]:
# | export
model_config = ConfigDict(extra="allow")

## Common Utilities

In [ ]:
# | exporti
def list2dict(variables: list):
    return {v.name: v for v in variables}


def normalize(v):
    """Utility function to convert a list to a dictionary"""
    return list2dict(v) if isinstance(v, list) else v


def normalize_t(type, v):
    """Utility function to convert a list of `type` to a dictionary"""
    return (
        list2dict(v)
        if isinstance(v, list) and all(isinstance(x, type) for x in v)
        else v
    )


def add_v(self, name, v):
    return getattr(self, name).update({v.name: v})

## Variable

In [ ]:
# | export
class Variable(BaseModel):
    model_config = model_config

    name: str = None
    description: str = None
    unit: str = None
    timerange: list[datetime] = None


class Variables(BaseModel):
    timerange: list[datetime] = None
    variables: dict[str, Variable] = None

    def add_variable(self, v):
        add_v(self, "variables", v)

## Dataset

In [ ]:
# | export
class Dataset(Variables):
    model_config = model_config

    name: str = None
    dataset: str = None  # ID of the dataset (like the `ProductKey` used in the SPASE)
    parameters: list[str] = None  # list of parameter names (core variables)
    ts: timedelta = None  # time resolution (all variables in one dataset have the same time resolution)

## Instrument and Mission

In [ ]:
# | export
class Instrument(BaseModel):
    model_config = model_config

    name: str
    datasets: list[Dataset] = Field(default_factory=list)

    type: str = None
    names_list: list[str] = None

    def add_dataset(self, dataset: Dataset):
        self.datasets.append(dataset)


class InstrumentSuite(BaseModel):
    model_config = model_config

    name: str
    names_list: list[str] = None
    type: str = "suite"
    instruments: list[Instrument] = Field(default_factory=list)
    # field_validator("instruments", mode="before")(normalize)


class Mission(BaseModel):
    model_config = model_config

    name: str
    """Name of the mission"""
    instruments: list[Instrument | InstrumentSuite] = Field(default_factory=list)
    datasets: list[Dataset] = Field(default_factory=list)
    names_list: list[str] = None

    def add_instrument(self, v):
        add_v(self, "instruments", v)